# Demo 2 - VTK conversions

In this notebook we use the [GNS-Science/eq_fault_geom](https://github.com/GNS-Science/eq-fault-geom) package to convert tsurf (from the 3D CFM) and ESRI shapefiles (from the 2D CFM) into VTK format which is simple and widely supported.

In [1]:
from pathlib2 import Path
import meshio
from eq_fault_geom import faultmeshio # this could be added to meshio ??\
import geopandas as gpd

## Convert CFM fault surfaces in hi resolution 

These are the **3D tsurf** files from CFM.

In [2]:
#convert the tsurf files from CFM into VTK format
inpath = Path("./data/CFM/tsurf")
outpath = Path("./tmp/hi-res")

#ensure outpath exists
outpath.mkdir(parents=True, exist_ok=True)

#iterate over the *.ts files, sor
for tsurfFile in inpath.glob('*.ts'):
    outFile = Path.joinpath(outpath, tsurfFile.stem).with_suffix('.vtk')
    tsurf = faultmeshio.tsurf(tsurfFile)
    meshio.write(str(outFile), tsurf.mesh, binary=False)

## Convert CFM 2D model to low-resolution stirling surfaces

The fault surface traces with dip and depth data are converted from the CFM 2D ESRI shapefiles to stirling surfaces as used in **opensha**.

In [3]:
# Read CFM shapefile into a geopandas dataframe
shp_df = gpd.GeoDataFrame.from_file("zip://./data/CFM/GIS/NZ_CFM_v0_3_180620.zip!NZ_CFM_v0_3_180620.shp")

In [4]:
from eq_fault_geom.geomio import cfm_shp_to_vtk as cv

In [5]:
flt1 = shp_df.FZ_Name.str.contains("Wellington Hutt Valley")
flt2 = shp_df.FZ_Name.str.contains("Wairarapa")
faults = shp_df[flt2|flt1]

faults = faults.sort_values("FZ_Name").reset_index(drop=True)
sorted_wgs = faults.to_crs(epsg=4326)

cv.output_path = Path("./tmp/lo-res")
cv.output_path.mkdir(parents=True, exist_ok=True)

for i, fault in sorted_wgs.iterrows():
    nztm_geometry_i = faults.iloc[i].geometry
    cv.create_stirling_vtk(fault, section_id=i, nztm_geometry=nztm_geometry_i)
